## PennyLane: Variational Quantum Algorithms
Annika Nel Jan 25

## Summary
VQA's use a parameterized circuit to solve a problem, where a classical algorithm optimises the parameters.

#### Method:
1. Choose an ansatz, U(θ) - a quantum circuit parametrized by a set of free parameters θ.
2. Measure an observable and compute cost function based on this measurement.
3. The free parameters θ are optimized using a classical computer that queries the quantum device.

#### How to choose an ansatz?
* Problem-inspired: make a good guess based on the problem you are trying to solve
* Problem-agnostic: general good guesses - more complex so more difficult to optimize

Consider the trade-off between _expressibility_ and _trainability_:
* Expressibility: quantifies the range of functions that the quantum circuit can describe. High entanglement + more parameters = high expressibility
* Trainability: how easy it is to find the optimal parameters. Harder with more complexity, i.e. higher expressibility.

#### Cost function
* Normally defined as the expectation value of some observable $\hat{M}$:
$$ C(\overrightarrow{\theta}) = \bra{0} U^{\dagger}(\overrightarrow{\theta}) \hat{M}  U(\overrightarrow{\theta}) \ket{0}  $$
* Gradient descent: We can find the gradient of a circuit by calculating the expectation value, then shifting the parameters and caculating it again

#### Limitations
* Optimization challenges: landscape of the cost function - many VQAs present a high-dimensional parameter space with Barren Plateaus (difficult for gradient descent)
* Complexity and scalability challenges: Difficult to characterize the time complexity of VQA's. Insights into time efficiency are often heuristic rather than definitive. Without mathematical proof of advantage and the ability to conduct large-scale experiments, potential for quantum advantage remains an open question.

## My questions
* How could Bayesian optimization come into play here? Can it even simulate the QC effectively?
* Since barren plateaus are a problem, could parallelized computational intelligence techniques provide any benefit?

--- 

In [1]:
import pennylane as qml
from pennylane import numpy as np
from pennylane.data.data_manager import params
import matplotlib.pyplot as plt

### Codercise V.1.1

In [2]:
n_bits = 4
dev = qml.device("default.qubit", wires=n_bits)

@qml.qnode(dev)
def basic_entangling_ansatz(observable,params):
    """Applies an ansatz with basic entanglement.

    Args:
        observable (qml.op): a pennylane operator whose expectation value we want to measure.
        params(np.array): an array with the trainable parameters of the ansatz. They have the shape of `qml.StronglyEntanglingLayers.shape(n_layers=1, n_wires=n_bits)`

    Returns:
        (np.tensor): a numpy tensor of 1 element corresponding to the expectation value of the given observable.
    """
    ##################
    # YOUR CODE HERE #
    ##################
    for i in range(n_bits):
        qml.RX(params[0][i],wires=i)
    for i in range(1, n_bits):
        qml.CNOT(wires=[i-1, i])
    qml.CNOT(wires=[n_bits-1, 0])

    return qml.expval(observable)


@qml.qnode(dev)
def strongly_entangling_ansatz(observable,params):
    """Applies an ansatz with moderate entanglement.

    Args:
        observable (qml.op): a pennylane operator whose expectation value we want to measure.
        params(np.array): an array with the trainable parameters of the ansatz. They have the shape of `qml.StronglyEntanglingLayers.shape(n_layers=1, n_wires=n_bits)`

    Returns:
        (np.tensor): a numpy tensor of 1 element corresponding to the expectation value of the given observable.
    """
    ##################
    # YOUR CODE HERE #
    ##################
    for i in range(n_bits):
        qml.Rot(params[0][i][0],params[0][i][1],params[0][i][2],wires=i)
    qml.CNOT(wires=[0, n_bits-1])
    for i in range(1, n_bits):
        qml.CNOT(wires=[i, i-1])

    return qml.expval(observable)


### Codercise V.1.2.a

In [3]:
dev = qml.device('default.qubit', wires=1)
# Define the quantum circuit with a parameterized RY gate
@qml.qnode(dev)
def circuit_to_differentiate(theta):
    """Quantum circuit we want to differentiate.

    Args:
        theta (float): parameter of the circuit with respect to which we differentiate.

    Returns:
        (np.tensor): a numpy tensor of 1 element corresponding to the expectation value of Z
    """
    ##################
    # YOUR CODE HERE #
    ##################
    qml.RY(theta, wires=0)
    return qml.expval(qml.Z(wires=0))

# Define the parameter-shift rule function
def parameter_shift_rule(theta):
    """Function that applies the parameter shift rule to `circuit_to_differentiate` with respect to the parameter theta.

    Args:
        theta (float): parameter of the circuit with respect to which we differentiate.

    Returns:
        (np.tensor): a numpy tensor of 1 element corresponding to the derivative of the circuit with respect to theta.
    """    
    ##################
    # YOUR CODE HERE #
    ##################
    f = lambda th: circuit_to_differentiate(th)

    return 0.5 * (f(theta + np.pi/2) - f(theta - np.pi/2))


### Codercise V.1.2.b

In [4]:
# Define the built-in parameter-shift rule function
def parameter_shift_rule_built_in(circuit, theta):
    """Function that applies the PennyLane built-in parameter shift rule to a specific circuit with respect to the parameter theta.

    Args:
        circuit (qml.QNode): quantum circuit we want to differentiate.
        theta (float): parameter of the circuit with respect to which we differentiate.

    Returns:
        (np.tensor): a numpy tensor of 1 element corresponding to the derivative of the circuit with respect to theta.
    """  
    ##################
    # YOUR CODE HERE #
    ##################
    return qml.gradients.param_shift(circuit)(theta)

# Define the built-in classical_jacobian function
def jacobian_built_in(circuit, theta):
    """Function that applies the PennyLane built-in jacobian method to a specific circuit with respect to the parameter theta.

    Args:
        circuit (qml.QNode): quantum circuit we want to differentiate.
        theta (float): parameter of the circuit with respect to which we differentiate.

    Returns:
        (np.tensor): a numpy tensor of 1 element corresponding to the derivative of the circuit with respect to theta.
    """  
    ##################
    # YOUR CODE HERE #
    ################## 
    return qml.jacobian(circuit)(theta)
    


### Codercise V.1.3.a

In [5]:
dev = qml.device('default.qubit', wires=1)
# Define the quantum circuit with a parameterized RY gate

@qml.qnode(dev)
def circuit(theta):
    """Quantum circuit we want to differentiate.

    Args:
        theta (float): parameter of the circuit with respect to which we differentiate.

    Returns:
        (np.tensor): a numpy tensor of 1 element corresponding to the expectation value of Z
    """
    ##################
    # YOUR CODE HERE #
    ##################
    qml.RY(theta, wires=0)
    return qml.expval(qml.Z(wires=0))

def gradient_descent_optimization(quantum_circuit, initial_theta, learning_rate, max_iterations):
    """
    Performs Gradient Descent optimization to find the optimal parameter theta
    for a quantum circuit to minimize its output expectation value.

    Args:
        quantum_circuit (qml.QNode): A quantum circuit that depends of a parameter.
        initial_theta (np.array): An array with the initial value of the trainable parameter theta.
        learning_rate (float): Learning rate for the gradient descent update.
        max_iterations (int): Maximum number of iterations for the optimization.

    Returns:
        (np.array): A numpy array of 1 element corresponding to the optimized parameter theta.
    """
    ##################
    # YOUR CODE HERE #
    ##################
    theta = initial_theta
    for _ in range(max_iterations):
        theta = theta - learning_rate * qml.gradients.param_shift(quantum_circuit)(theta)
    return theta
    

optimized_theta=gradient_descent_optimization(circuit, np.array(0.1, requires_grad=True), 0.3, 50)
print(f"Optimized theta using Gradient Descent: {optimized_theta}")
print(f"Expectation value of quantum circuit at optimized theta: {circuit(optimized_theta)}")

Optimized theta using Gradient Descent: 3.141590464104736
Expectation value of quantum circuit at optimized theta: -0.9999999999976031


### Codercise V.1.3.b

In [6]:
def gradient_descent_optimization_built_in(quantum_circuit,initial_theta, learning_rate, max_iterations):
    """
    Implements Gradient Descent optimization method of PennyLane to find the optimal parameter theta
    for a quantum circuit to minimize its output expectation value.

    Args:
        quantum_circuit (qml.QNode): A quantum circuit that depends of a parameter.
        initial_theta (np.array): An array with the initial value of the trainable parameter theta.
        learning_rate (float): Learning rate for the gradient descent update.
        max_iterations (int): Maximum number of iterations for the optimization.

    Returns:
        (np.array): A numpy array of 1 element corresponding to the optimized parameter theta.
    """
    ##################
    # YOUR CODE HERE #
    ##################
    optimizer = qml.GradientDescentOptimizer(learning_rate)
    theta = initial_theta

    for _ in range(max_iterations):
        theta = optimizer.step(quantum_circuit, theta)

    return theta
    
optimized_theta=gradient_descent_optimization_built_in(circuit,np.array(0.1,requires_grad=True), 0.3, 50)
print(f"Optimized theta using built-in Gradient Descent: {optimized_theta}")
print(f"Expectation value of quantum circuit at optimized theta: {circuit(optimized_theta)}")



Optimized theta using built-in Gradient Descent: 3.141590464104736
Expectation value of quantum circuit at optimized theta: -0.9999999999976031


### Codercise V.1.4

In [9]:
@qml.qnode(dev)
def strongly_entangling_ansatz(observable, params):
    """Applies an ansatz with moderate entanglement.

    Args:
        observable (qml.op): a pennylane operator whose expectation value we want to measure.
        params(np.array): an array with the trainable parameters of the ansatz. They have the shape of `qml.StronglyEntanglingLayers.shape(n_layers=1, n_wires=n_bits)`

    Returns:
        (np.tensor): a numpy tensor of 1 element corresponding to the expectation value of the given observable.
    """
    ##################
    # YOUR CODE HERE #
    ##################
    for i in range(n_bits):
        qml.Rot(params[0][i][0],params[0][i][1],params[0][i][2],wires=i)
    qml.CNOT(wires=[0, n_bits-1])
    for i in range(1, n_bits):
        qml.CNOT(wires=[i, i-1])

    return qml.expval(observable)

def cost_function(observable, observableparams):
    """Computes the cost function we want to minimize.

    Args:
        observable (qml.Hamiltonian): a pennylane Hamiltonian whose expectation value we want to measure.
        params(np.array): an array with the trainable parameters of the ansatz.

    Returns:
        (np.tensor): a numpy tensor of 1 element corresponding to the cost function value
    """
    ##################
    # YOUR CODE HERE #
    ##################
    return strongly_entangling_ansatz(observable, observableparams)
    

def optimizer(observable, params, learning_rate=0.1, steps=100):
    """Updates the parameters to minimize the cost function value.

    Args:
        observable (qml.Hamiltonian): a pennylane Hamiltonian whose expectation value we want to measure.
        params(np.array): an array with the trainable parameters of the ansatz.

    Returns:
        (np.array): an array with the optimized trainable parameters.
    """
    ##################
    # YOUR CODE HERE #
    ##################
    optimizer = qml.GradientDescentOptimizer(learning_rate)
    optimized_params = params.copy()
    cost = lambda p: cost_function(observable, p)

    for _ in range(steps):
        optimized_params = optimizer.step(cost, optimized_params)

    return optimized_params
    

In [10]:
num_layers = 1
num_qubits = 4

dev = qml.device('default.qubit', wires=num_qubits)
shape = qml.StronglyEntanglingLayers.shape(n_layers=num_layers, n_wires=num_qubits)
initial_params = np.random.randn(*shape, requires_grad=True)

M = qml.Z(wires=0) @ qml.Z(wires=1) @ qml.Z(wires=2) @ qml.Z(wires=3)
print("Initial cost:", cost_function(M, initial_params))

opt_params = optimizer(M, initial_params)

print("Optimized parameters:\n", opt_params)
print("Final cost:", cost_function(M, opt_params))

Initial cost: 0.9851462128432489
Optimized parameters:
 [[[ 0.28343882  3.14079459  0.47582266]
  [ 1.24979368 -1.1441016  -1.03249956]
  [-0.79655982  1.65087592  0.61724533]
  [ 0.64593448  1.89518846  0.55138141]]]
Final cost: -0.9999996815447184


---